In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('data.csv')
data

,1102,710,-655,0
0,145,-105,882,0
1,62,305,741,0
2,78,50,792,0
3,-58,19,1031,0
4,-109,-35,1082,0
...,...,...,...,...
132698,-305,474,659,9
132699,-309,478,670,9
132700,-325,494,674,9
132701,-325,509,678,9


In [2]:
df=pd.DataFrame(data)
df.columns=['x-axis','y-axis','z-axis','number']
df

,x-axis,y-axis,z-axis,number
0,145,-105,882,0
1,62,305,741,0
2,78,50,792,0
3,-58,19,1031,0
4,-109,-35,1082,0
...,...,...,...,...
132698,-305,474,659,9
132699,-309,478,670,9
132700,-325,494,674,9
132701,-325,509,678,9


In [3]:
df['number'].value_counts()

44    13315
3     13300
8     13296
2     13280
5     13279
0     13270
6     13257
7     13243
1     13233
9     13230
Name: number, dtype: int64

In [5]:
df['number'] = df['number'].replace(44,4)
df['number'].value_counts()


4    13315
3    13300
8    13296
2    13280
5    13279
0    13270
6    13257
7    13243
1    13233
9    13230
Name: number, dtype: int64

##Standardized data

In [8]:
from sklearn.preprocessing import StandardScaler
x=df[['x-axis','y-axis','z-axis']]
y=df['number']
scaler=StandardScaler()
X=scaler.fit_transform(x)


In [9]:
scaled_X=pd.DataFrame(data=X,columns=['x-axis','y-axis','z-axis'])
scaled_X['number']=y.values

scaled_X

,x-axis,y-axis,z-axis,number
0,0.615454,-0.445042,0.223163,0
1,0.318892,0.355091,-0.123441,0
2,0.376060,-0.142553,0.001926,0
3,-0.109873,-0.203051,0.589434,0
4,-0.292097,-0.308434,0.714801,0
...,...,...,...,...
132698,-0.992413,0.684902,-0.325013,9
132699,-1.006705,0.692708,-0.297973,9
132700,-1.063873,0.723933,-0.288140,9
132701,-1.063873,0.753206,-0.278307,9


In [10]:
df

,x-axis,y-axis,z-axis,number
0,145,-105,882,0
1,62,305,741,0
2,78,50,792,0
3,-58,19,1031,0
4,-109,-35,1082,0
...,...,...,...,...
132698,-305,474,659,9
132699,-309,478,670,9
132700,-325,494,674,9
132701,-325,509,678,9


In [11]:
import scipy.stats as stats

In [12]:
fs=20
frame_size=fs*9
hop_size=fs*2

In [ ]:
def get_frames(df,frame_size,hop_size):
    